run_qNN_notebook.ipynb

Notebook that can run the qNN with a single qubit

Dependencies:
- Uses numpy to get random values
- Uses qNN_functions to get qNN necessary functions 

Since:
- 02/2025

Authors:
- Pedro C. Delbem. <pedrodelbem@usp.br>

#imports

In [2]:
import numpy as np #type: ignore
import qNN_functions as sqNN

#generate qNN circuit

In [3]:
#declare the input
input = [0,0]

In [4]:
#generate 3 random angles
angles = np.random.rand(3)*np.pi
angles

array([3.06501917, 1.36987186, 2.13776327])

In [5]:
#generate circuit
single_qubit_qNN = sqNN.generate_qNN_circuit(input,angles)
single_qubit_qNN.get_current_circuit().draw()

┌───┐┌────────────┐┌────────────┐ ░ ┌─┐
     q: ┤ H ├┤ Rz(2.1378) ├┤ Rx(2.1378) ├─░─┤M├
        └───┘└────────────┘└────────────┘ ░ └╥┘
   c: 1/═════════════════════════════════════╬═
                                             ║ 
meas: 1/═════════════════════════════════════╩═
                                             0

create the circuit to the quantum neuron with 3 angles whose first 2 angles multiply the XOR inputs and the third value is the bias that is added to the multiplications
rz angle expression: angle1*input1 + angle3
rx angle expression: angle2*input2 + angle3

functions:
def neuron(qc,input1_value,input2_value,weight1,weight2,weight3,qbit_index,classical_bit_index):
    """
    Applies a quantum neuron operation to the given quantum circuit.

    Parameters:
    qc (QuantumCircuit): The quantum circuit to which the quantum neuron operation is applied.
    input1_value (float): The value of the first input of the neuron.
    input2_value (float): The value of the second input of the neuron.
    weight1 (float): The weight of the first input of the neuron.
    weight2 (float): The weight of the second input of the neuron.
    weight3 (float): The bias of the neuron.
    qbit_index (int): The index of the qbit to which the quantum neuron operation is applied.
    classical_bit_index (int): The index of the classical bit to which the quantum neuron operation is applied.
    """

    qc.h(qbit_index)

    qc.rz(input1_value*weight1+weight3,qbit_index)
    qc.rx(input2_value*weight2+weight3,qbit_index)

def add_neuron(self,input1_value,input2_value,weight1,weight2,weight3,qbit_index,classical_bit_index):
    """
    Add a quantum neuron operation to the current quantum circuit.

    Parameters:
    input1_value (float): The value of the first input of the neuron.
    input2_value (float): The value of the second input of the neuron.
    weight1 (float): The weight of the first input of the neuron.
    weight2 (float): The weight of the second input of the neuron.
    weight3 (float): The bias of the neuron.
    qbit_index (int): The index of the qbit to which the quantum neuron operation is applied.
    classical_bit_index (int): The index of the classical bit to which the quantum neuron operation is applied.
    """
    neuron(self._qc,input1_value,input2_value,weight1,weight2,weight3,qbit_index,classical_bit_index)


def generate_qNN_circuit(inputs,parameters):
    """
    Generate a quantum neural network circuit (qNN) with a single neuron.

    Parameters:
    input1_value (float): The value of the first input of the neuron.
    input2_value (float): The value of the second input of the neuron.
    parameters (list of floats): The parameters of the neuron, in order: first input weight, second input weight, bias.

    Returns:
    The qNN circuit (current_circuit).
    """
    qNN = current_circuit(1,1) #create the qNN circuit
    qNN.add_neuron(*inputs, *parameters, 0, 0) #add the neuron
    qNN.get_current_circuit().measure_all()

    return qNN

#evaluate qNN circuit

In [6]:
counts = single_qubit_qNN.evaluate(number_of_shots=1024, number_of_runs=1)

uses the evaluate structure, from the current circuit class, to obtain results from the circuit

function:
def evaluate(self, number_of_shots = 1024, number_of_runs = 100):
    """
    Evaluate a quantum circuit (XOR candidate) and return the counts (histogram of the outputs).

    Parameters:
    quantum_circuit (QuantumCircuit): The quantum circuit to be evaluated.
    number_of_shots (int): The number of shots to be used in the evaluation.
    number_of_runs (int): The number of times the quantum circuit is run.

    Returns:
    list: A list of dictionaries, where each dictionary represents the counts of the outputs of the quantum circuit.
    """

    #sample results with severals runs, each with several shots
    sampler = StatevectorSampler()
    #create jobs list
    jobs = []
    
    #run the circuit several times
    for _ in range(number_of_runs):

        #run the circuit
        job = sampler.run([(self._qc)], shots = number_of_shots)
        #append the job to the jobs list
        jobs.append(job)

    #create the counts list
    counts = []

    #get and show raw results - counts
    for job in jobs:

        #get the data
        data_pub = job.result()[0].data # 'pub' refers to Primitive Unified Bloc
        job_counts = data_pub.meas.get_counts()

        #append the counts to the counts list
        counts.append(job_counts)

    #return the counts list
    return counts

#compute the error

In [7]:
#define inputs and the expected outputs
inputs = [[0,0],[0,1],[1,0],[1,1]]
expected_outputs = [str(input1*input2) for input1,input2 in inputs]

In [8]:
error = sqNN.compute_total_error(inputs,expected_outputs,angles)
error

0.577880859375

computes the circuit error when simulating the XOR gate for given angles

functions:
def compute_error(counts,expected_output):
    """
    Compute the error between the actual outputs and the expected outputs.

    Parameters:
    counts (list of dictionaries): The counts of the outputs of the quantum circuit.
    expected_outputs (list of floats): The expected outputs of the quantum circuit.

    Returns:
    The error (float).
    """

    #compute number of shots
    number_of_shots = sum(counts[0].values())

    #initialize error with 0
    error = 0

    #compute error for each count
    for count in counts:
        if expected_output in count:
            error = number_of_shots - count[expected_output]
        else:
            error = number_of_shots

    #normalize error
    error = error/number_of_shots

    #return error
    return error

def compute_total_error(inputs,expected_outputs,parameters,number_of_runs=1):
    """
    Compute the total error for a set of inputs and expected outputs.

    Parameters:
    inputs (list of lists): A list containing pairs of input values for the neuron.
    expected_outputs (list of floats): A list of expected output values for each input pair.
    parameters (list of floats): The parameters of the neuron, including weights and bias.
    number_of_runs (int): The number of times the circuit is run.

    Returns:
    The total error (float) across all input pairs.
    """

    #initialize total error
    total_error = 0

    #apply qNN circuit to each input
    for interation in range(len(inputs)):

        qNN_circuit = generate_qNN_circuit(inputs[interation],parameters) #generate circuit
        counts = qNN_circuit.evaluate(number_of_runs=number_of_runs) #run circuit
        total_error += compute_error(counts,expected_outputs[interation]) #add error

    #normalize total error
    total_error = total_error/len(inputs)

    #return total error
    return total_error

#exaustive search

In [9]:
final_parameters, final_error = sqNN.exaustive_search(inputs,expected_outputs)
final_parameters, final_error

([np.float64(0.0),
  np.float64(-1.5707963267948966),
  np.float64(1.5707963267948966)],
 0.2470703125)

do a exaustive search by varying the angles in order to obtain the angle with the lowest error for the XOR gate - obtaning the angles and the error

function:
def exaustive_search(inputs,expected_outputs,grid_grain=5,number_of_runs=1):
    """
    Perform an exaustive search of the parameter space to find the optimal parameters for the given inputs and expected outputs.

    Parameters:
    inputs (list of lists): A list containing pairs of input values for the neuron.
    expected_outputs (list of floats): A list of expected output values for each input pair.
    grid_grain (int): The number of points in the grid to search.
    number_of_runs (int): The number of times the circuit is run.

    Returns:
    The optimal parameters (list of floats) and the total error (float) of the optimal parameters.
    """

    #initialize final error
    final_error = 1

    #initialize final parameters
    final_parameters = [0,0,0]

    #exaustive search
    for weight1 in np.linspace(-np.pi, np.pi, grid_grain):
        for weight2 in np.linspace(-np.pi, np.pi, grid_grain):
            for weight3 in np.linspace(-np.pi, np.pi, grid_grain):

                #compute total error
                parameters = [weight1, weight2, weight3]
                current_error = compute_total_error(inputs,expected_outputs,parameters,number_of_runs=number_of_runs)

                #update final error
                if current_error < final_error:
                    final_error = current_error
                    final_parameters = parameters

    #return final parameters
    return final_parameters, final_error